# NOSQL AND QUERYING JSON AND XML/HTML OBJECTS/FILES

**by Serhat Çevikel**

## Working with JSON Objects

### Querying JSON with JQ

- Or better, we will use a handy tool called "jq" for this purpose:

In [1]:
echo $record1 | jq .

- What is more powerful about JSON format is that you can embedded documents inside other ones: (we print the data here in three visual formats)

- Multiple documents can be "collected" inside "collections":

- A "collection" in NoSQL terminology is analogous to a "table" in the RDBMS jargon. A collection is a collection of similar items (or documents with similar key-value pairs)

### A real json database example: UN COMTRADE

In [ ]:
[
    {
        "type": "COMMODITIES",
        "freq": "MONTHLY",
        "px": "HS",
        "r": "76",
        "rDesc": "Brazil",
        "ps": "201607",
        "TotalRecords": 126827,
        "IsOriginal": 1,
        "IsPartnerDetail": 1,
        "UploadTime": "2016-08-05T00:00:00"
    },
    {
        "type": "COMMODITIES",
        "freq": "MONTHLY",
        "px": "HS",
        "r": "579",
        "rDesc": "Norway",
        "ps": "201607",
        "TotalRecords": 110618,
        "IsOriginal": 1,
        "IsPartnerDetail": 1,
        "UploadTime": "2016-08-30T00:00:00"
    },
    ...
]

bash: [: missing `]'


- We will be using a part of the UN COMTRADE database:

[UN COMTRADE](https://comtrade.un.org/)

UN COMTRADE is the widest and most comprehensive database on international trade:

- 250+ reporter countries
- 290+ partner countries
- 6500+ commodity codes
- 50+ of history
- Both imports and exports
- Both values and quantities!

Let's first save the path:

In [3]:
datadir=~/data
comtrade=$datadir/comtrade_s1

In [4]:
ls $comtrade

2010   classificationS1.json  reporterAreas.json
2010e  partnerAreas.json      tradeRegimes.json


- classificationS1.json lists the item classification according to SITC1 method
- reporterAreas.json and partnerAreas.json lists the countries and their respective codes
- data files are under 2010

#### Reporters

In [5]:
jq . $comtrade/partnerAreas.json | head -20

{
  "more": false,
  "results": [
    {
      "id": "all",
      "text": "All"
    },
    {
      "id": "0",
      "text": "World"
    },
    {
      "id": "4",
      "text": "Afghanistan"
    },
    {
      "id": "472",
      "text": "Africa CAMEU region, nes"
    },
    {
Error: writing output failed: Broken pipe


In [6]:
partnerlist=$(jq -r '.results[] | "\(.text)\t\(.id)"' $comtrade/partnerAreas.json | \
xargs -0 -i echo -e "{}")

reporterlist=$(jq -r '.results[] | "\(.text)\t\(.id)"' $comtrade/reporterAreas.json | \
xargs -0 -i echo -e "{}")

echo "$partnerlist" |  column -s $'\t' -t | head -20

All                                           all
World                                         0
Afghanistan                                   4
Africa CAMEU region, nes                      472
Albania                                       8
Algeria                                       12
American Samoa                                16
Andorra                                       20
Angola                                        24
Anguilla                                      660
Antarctica                                    10
Antigua and Barbuda                           28
Areas, nes                                    899
Argentina                                     32
Armenia                                       51
Aruba                                         533
Australia                                     36
Austria                                       40
Azerbaijan                                    31
Bahamas                                       44


In [8]:
echo "$reporterlist" | awk -F "\t" '$1=="Azerbaijan"{print $2}'

31


In [9]:
echo "$partnerlist" | awk -F "\t" '$1=="Turkey"{print $2}'

792


In [11]:
Azerbaijan=$(for file in $comtrade/2010e/*_31_792_*.json;
do
    jq -r '.dataset[] | select(.IsLeaf==1) |
"\(.rtTitle)\t\(.ptTitle)\t\(.TradeValue)\t\(.rgCode)\t\(.cmdCode)\t\(.cmdDescE)\t\(.IsLeaf)"' $file | \
xargs -0 -i echo -e "{}";
done)


echo "$Azerbaijan" | sort -t $'\t' -nr -k 3 | column -s $'\t' -t  | head -20

Azerbaijan  Turkey  41925916  1  6911   Fin.structural parts & structures of iron steel  1
Azerbaijan  Turkey  37807399  1  5542   Surface acting agents and washing preparations   1
Azerbaijan  Turkey  24866267  1  64299  Other art.of paper/pulp/board/cellulose wadding  1
Azerbaijan  Turkey  22078974  2  5812   Products of polymerization and copolymerization  1
Azerbaijan  Turkey  21951975  1  5812   Products of polymerization and copolymerization  1
Azerbaijan  Turkey  21527353  1  82109  Furniture and parts thereof,n.e.s.               1
Azerbaijan  Turkey  21121289  1  8930   Articles of artif.plastic materials,n.e.s.       1
Azerbaijan  Turkey  14449109  1  7321   Passenger motor cars, other than buses           1
Azerbaijan  Turkey  13374480  1  71931  Lifting & loading machinery                      1
Azerbaijan  Turkey  13033020  1  5811   Prods of condensation, polycond. & polyaddition  1
Azerbaijan  Turkey  12636529  1  71839  Other food processing machines                   1

In [16]:
jq -r '.results[] | select((.id|test("^\\d+$")) and (.text|test("(?i)Machine"))) |
select((.id|tonumber < 1000) and (.id|tonumber > 99)) | .text' \
$comtrade/classificationS1.json | \
sed 's/ - /\t/g' | column -s $'\t' -t

695  Tools for use in the hand or in machines
711  Power generating machinery, other than electric
712  Agricultural machinery and implements
714  Office machines
715  Metalworking machinery
717  Textile and leather machinery
718  Machines for special industries
719  Machinery and appliances non electrical parts
722  Electric power machinery and switchgear
729  Other electrical machinery and apparatus


In [18]:
jq -r '.dataset[] | select(.cmdCode == "695" and .rgCode == 2) |
"\(.ptTitle)\t\(.TradeValue)\t\(.TradeQuantity)"' $comtrade/2010e/2010_792_100_s1.json | \
xargs -0 -i echo -e "{}"

Bulgaria	1174870	148162

